# รายชื่อสมาชิกกลุ่ม

| Student ID  | Name              |
| ----------- | ----------------- |
| 64090500432 | ตวงทรัพย์ รัตนไทย     |
| 64090500436 | นันทวัฒน์ สุกก่ำ     |
| 64090500449 | วิชชากร เจริญยิ่ง     |
| 64090500450 | สินธร วิลเค่     |



# Library

In [ ]:
%pip install pandas
%pip install openpyxl
%pip install lxml
%pip install seaborn
%pip install scipy

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.stats as stats

# Read File

In [ ]:
df_accidents = pd.read_excel('../accidents.xlsx')
df_covid19 = pd.read_csv('../covid19.csv')
df_gdp_per_capita_growth = pd.read_csv('../gdp_per_capita_growth.csv')
df_item_categories = pd.read_csv('../item_categories.csv')
df_items = pd.read_csv('../items.csv')
df_sales = pd.read_csv('../sales.csv')
df_shops = pd.read_csv('../shops.csv')

# โจทย์

*** ข้อ 1-6 คำนวณเฉพาะสินค้าที่ขายได้โดยไม่นับรวมสินค้าที่ถูกคืน ***

## 1. ในทีนี้ เราจะไม่สนใจการคืนสินค้า ดังนั้น ให้ตัดรายการข้อมูล sales.csv ที่มีค่า item_cnt_day หรือ item_price น้อยกว่า 0 และแสดงสรุปข้อมูลให้เห็นในภาพรวมว่าตัดออกไปแล้ว (1 คะแนน)

ก่อนตัดข้อมูล: เรียงลำดับให้เห็นว่ามีข้อมูล item_cnt_day น้อยกว่า 0 หรือ item_price น้อยกว่า 0

In [ ]:
df_sales.sort_values(by=['item_cnt_day', 'item_price'], ascending=[True, True])

หลังตัดข้อมูล: เรียงลำดับให้เห็นว่ามีข้อมูล item_cnt_day น้อยกว่า 0 หรือ item_price น้อยกว่า 0

In [ ]:
sales_filtered = df_sales[(df_sales['item_cnt_day'] >= 0) & (df_sales['item_price'] >= 0)].copy()
sales_filtered.sort_values(by=['item_cnt_day', 'item_price'], ascending=[True, True])

## 2. ทุกร้านค้ารวมกันขายสินค้าไปได้กี่ชนิดสินค้า (1 คะแนน)

In [ ]:
print('จำนวนสินค้าทั้งหมดที่ขายไป:', sales_filtered['item_id'].nunique())

## 3. สินค้าชนิดใดที่ขายได้มากชิ้นที่สุดในแต่ละร้าน โดยแสดงชื่อร้าน ชื่อสินค้านั้น และจำนวนทีร้านนั้นขายสินค้านั้นได้ โดยแสดงมาเพียง 10 ร้านค้าที้มีจำนวนชิ้นที่ขายได้สูงสุดพอ (4 คะแนน) (ทุกข้อไม่ต้องมีเส้นตาราง)

In [ ]:
sales_shop = sales_filtered.merge(df_shops, how='left', on='shop_id')
sales_shop_item = sales_shop.merge(df_items, how='left', on='item_id')

item_count_by_shop = sales_shop_item.groupby(['shop_id', 'item_id'])['item_cnt_day'].sum().reset_index(name='item_count')

item_count_by_shop['shop_id'] = item_count_by_shop['shop_id'].map(df_shops.set_index('shop_id')['shop_name'])
item_count_by_shop = item_count_by_shop.rename(columns={
  'shop_id': 'Shop Name',
  'item_id': 'Most sold item ID',
  'item_count': 'Number of items sold'})

item_count_by_shop.sort_values(by=['Number of items sold'], ascending=False).head(10)

## 4. แสดงค่าเฉลี่ย, ค่ามัธยฐาน, ค่าสูงสุดและค่าต่ำสุด ของราคาสินค้าของแต่ละร้านค้า โดยเรียงลำดับตามชื่อของร้านค้าจาก A ไป Z รวมทั้งจำนวนสินค้าที่ร้านนั้นมีขายด้วย แสดงผลลัพธ์ดังตารางต่อไปนี้ (4 คะแนน)

In [ ]:
sales_shop = sales_filtered.merge(df_shops, on='shop_id')

sales_shop_stats = sales_shop.groupby(['shop_id', 'shop_name']).agg({
    'item_price': ['mean', 'median', 'max', 'min'],
    'item_id': 'nunique'
}).reset_index()

sales_shop_stats = sales_shop_stats.drop(columns=['shop_id'], axis=1, level=0)
sales_shop_stats.columns = sales_shop_stats.columns.droplevel(level=0)
sales_shop_stats = sales_shop_stats.rename(columns={
    (''): 'Shop Name',
    ('mean'): 'Average',
    ('median'): 'Median',
    ('max'): 'Highest',
    ('min'): 'Lowest',
    ('nunique'): 'Number of items'
})

sales_shop_stats = sales_shop_stats[['Shop Name', 'Number of items', 'Average', 'Median', 'Highest', 'Lowest']]
sales_shop_stats = sales_shop_stats.sort_values(by=['Shop Name']).reset_index(drop=True)
sales_shop_stats

## 5. ยอดขายคำนวณได้จากราคาสินค้า * จำนวนที่ขายได้ ในที่นี้ ให้นำเสนอข้อมูลในรูปตาราง เพื่อรายงานสรุปดังตารางต่อไปนี้ โดยที่

### 5.1 แต่ละร้านค้าขายสินค้าแต่ละหมวด (category) รวมได้มูลค่าเท่าใด (คอลัมน์ที่ 3) (5 คะแนน)

In [ ]:
sales_shop_item_category = sales_shop_item.merge(df_item_categories, on='item_category_id')
sales_shop_item_category['Sales'] = sales_shop_item_category['item_price'] * sales_shop_item_category['item_cnt_day']

total_info = sales_shop_item_category.groupby(['shop_name', 'item_category_name'])['Sales'].sum().reset_index()
total_info.rename(columns={
  'shop_name': 'Shop Name',
  'item_category_name': 'Product Category',
  'Sales': 'Sales'
  }, inplace=True)

pd.options.display.float_format = '{:.2f}'.format
total_info

### 5.2 โดยให้เรียงลำดับด้วยชื่อร้านค้า (Shop Name) (จากน้อยไปมาก) แล้วยอดขายของแต่ละหมวดสินค้า (คอลัมน์ที่ 3) จากมากไปน้อย (3 คะแนน)

In [ ]:
total_info.sort_values(by=['Shop Name', 'Sales'], ascending=[True, False], inplace=True)
total_info

## 6. ในข้อนี้ ให้พิจารณาร้านค้าที่มียอดขายรวมสูงสุดเท่านั้น

### 6.1 ให้แสดงและวิเคราะห์ยอดขายของร้านโดยแยกเป็นรายเดือน และรายวันของสัปดาห์ (Day of week) (5 คะแนน)

In [ ]:
sales_shop_item_category['date'] = pd.to_datetime(sales_shop_item_category['date'], format='%d.%m.%Y')
sales_shop_item_category['Day of week'] = sales_shop_item_category['date'].dt.day_name()
sales_shop_item_category['number of month'] = sales_shop_item_category['date'].dt.month
sales_shop_item_category['number of day'] = sales_shop_item_category['date'].dt.dayofweek
sales_shop_item_category['Month'] = sales_shop_item_category['date'].dt.strftime('%B')

top_sale = sales_shop_item_category.groupby(['shop_id','shop_name'])['Sales'].sum().reset_index()
top_sale.sort_values(by=['Sales'], ascending=False, inplace=True)

shop_top_sale = sales_shop_item_category[sales_shop_item_category['shop_id'] == top_sale['shop_id'].iloc[0]]
shop_top_sale_name = top_sale['shop_name'].iloc[0]

รายเดือน

In [ ]:
month_sales = shop_top_sale.groupby(['number of month','Month'])['Sales'].sum().reset_index()
month_sales.drop(columns=['number of month'])

รายวัน

In [ ]:
day_sales = shop_top_sale.groupby(['number of day','Day of week'])['Sales'].sum().reset_index()
day_sales.drop(columns=['number of day'])

### 6.2 นำข้อมูลในข้อ 6.1 มาแสดงเป็นกราฟ (Chart) อย่างเหมาะสม (2 คะแนน)

กราฟรายเดือน

In [ ]:
plt.figure(figsize=(12, 6))

bars = plt.bar(month_sales['Month'], month_sales['Sales'], color='skyblue')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:,.2f}', ha='center', va='bottom', fontsize=9, color='black')

plt.grid(axis='y', alpha=0.5)
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: '{:,.0f}'.format(x)))
plt.xticks(rotation=45)
plt.title(f'{shop_top_sale_name}: Monthly Sales Report')
plt.xlabel('Month', labelpad=30)
plt.ylabel('Sales', labelpad=40)
plt.tight_layout()

plt.show()

กราฟรายวัน

In [ ]:
plt.figure(figsize=(12, 6))

bars = plt.bar(day_sales['Day of week'], day_sales['Sales'], color='skyblue')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:,.2f}', ha='center', va='bottom', fontsize=9, color='black')

plt.grid(axis='y', alpha=0.5)
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: '{:,.0f}'.format(x)))
plt.xticks(rotation=45)
plt.title(f'{shop_top_sale_name}: Daily Sales Report')
plt.xlabel('Day of week', labelpad=30)
plt.ylabel('Sales', labelpad=40)
plt.tight_layout()

plt.show()

### 6.3 จากนั้นให้แสดงกราฟ (Chart) เพื่อให้เห็นว่าแต่ละวันของสัปดาห์ (Day of week) ขายได้คิดเป็นสัดส่วนเท่าใด (2 คะแนน)

In [ ]:
day_sales['Percentage'] = day_sales['Sales'] / day_sales['Sales'].sum() * 100

plt.figure(figsize=(12, 6))

bars = plt.bar(day_sales['Day of week'], day_sales['Percentage'], color='skyblue')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:,.2f}%', ha='center', va='bottom', fontsize=9, color='black')

plt.grid(axis='y', alpha=0.5)
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: '{:,.0f}%'.format(x)))
plt.xticks(rotation=45)
plt.title(f'{shop_top_sale_name}: Daily Sales Report')
plt.xlabel('Day of week', labelpad=30)
plt.ylabel('Sales (%)', labelpad=40)
plt.tight_layout()

plt.show()

## 7. จากข้อมูล gdp_per_capita_growth.csv หรือ อัตราการเติบโตของผลิตภัณฑ์มวลรวมต่อประชากร ที่ให้มาของ 220 ประเทศตั้งแต่ปี 1980 ถึง 2019 ให้นำเสนอมูลค่าผลิตภัณฑ์มวลรวมต่อประชากรของ China, Vietnam, India, Thailand, Singapore, Germany, France, USA, Japan, Madagascar, และ Congo โดยกำหนดค่าตั้งต้นเท่ากันเป็น 100 ที่ปีฐาน 1980 และให้นำเสนอเป็นกราฟที่สื่อความหมายคล้ายกับรูปต่อไปนี้ (แต่ควรจะดีกว่า) (6 คะแนน)

-90µ = -90 * 10^-6

In [ ]:
selected_years = [col for col in df_gdp_per_capita_growth.columns if col.isdigit() and int(col) >= 1980]
selected_countries = ['China', 'Vietnam', 'India', 'Thailand', 'Singapore', 'Germany', 'France', 'USA', 'Japan', 'Madagascar', 'Congo']

plot_data = {'Year': selected_years}

for country in selected_countries:
    data_values = df_gdp_per_capita_growth[df_gdp_per_capita_growth['country'] == country][selected_years].values[0]

    data_values = [data.replace('−', '-').replace('µ', 'e-6') if isinstance(data, str) else data for data in data_values]
    data_values = pd.to_numeric(data_values, errors='coerce')

    data_values[0] = 100
    for i in range(1, len(data_values)):
        data_values[i] = data_values[i - 1] + ((data_values[i] / 100) * data_values[i - 1])
    
    plot_data[country] = data_values

plot_data = pd.DataFrame(plot_data)

กราฟ

In [ ]:
plt.figure(figsize=(18, 9))

lines = plt.plot(plot_data['Year'], plot_data.drop(columns=['Year']))
len_year = len(plot_data['Year'].unique()) // 3
xtrick = [int(i * 3) for i in range(len_year + 1)]
ytrick = [i * 100 for i in range(11)]

plt.xlim(0, len(plot_data['Year']) - 1)

plt.grid(axis='y', alpha=0.5)
plt.gca().set_xticks(xtrick)
plt.gca().set_yticks(ytrick)  # Set y-axis ticks every 100%
plt.title('Growth Comparison Over Years')
plt.xlabel('Year', labelpad=30)
plt.ylabel(f'Growth% (based on {plot_data["Year"].iloc[0]})', labelpad=40)
plt.legend(selected_countries)
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(100))

for i in range(len(xtrick)):
    plt.axvline(x=xtrick[i], color='gray', linestyle='--', linewidth=0.8)

for line, country in zip(lines, selected_countries):
    final_value = line.get_ydata()[-1]
    plt.text(len(plot_data['Year']) - 0.7, final_value, f'{country}: {final_value:.2f}%', ha='left', va='center', color=line.get_color())

plt.show()

## 8. พิจารณาข้อมูลยอดผู้เสียชีวิตสะสม covid19.csv

### 8.1 ในกลุ่มประเทศทีมีผู้เสียชีวิต (เฉพาะวันล่าสุด) มากที่สุด 20 ประเทศ ให้นำเสนอสัดส่วนของจำนวนผู้ป่วยในแต่ละประเทศเมื่อเทียบกับทั้งหมด (20 ประเทศนี้) ทั้งนี้ให้รวมประเทศลำดับที่ 11-20 เป็นอื่นๆ (Others) (4 คะแนน)

In [ ]:
latest_covid19 = df_covid19[['Country/Region', df_covid19.columns[-1]]].copy()
latest_covid19.rename(columns={latest_covid19.columns[-1]: 'Total Case'}, inplace=True)
latest_covid19 = latest_covid19.sort_values(by='Total Case', ascending=False).head(20).reset_index(drop=True)
latest_covid19['Total Case'] = pd.to_numeric(latest_covid19['Total Case'], errors='coerce')
latest_covid19['Percentage'] = latest_covid19['Total Case'] / latest_covid19['Total Case'].sum() * 100

grouped_row_sum = {
  'Country/Region': 'Others',
  'Total Case': latest_covid19.iloc[10:]['Total Case'].sum(),
  'Percentage': latest_covid19.iloc[10:]['Percentage'].sum()
} 
grouped_row_sum = pd.DataFrame(grouped_row_sum, index=[0])

latest_covid19.drop(index=latest_covid19.index[10:], inplace=True)
latest_covid19 = pd.concat([latest_covid19, grouped_row_sum], ignore_index=True)

pd.options.display.float_format = '{:,.2f}'.format
latest_covid19

กราฟ

In [ ]:
latest_covid19.sort_values(by='Total Case', ascending=False, inplace=True)

fig, ax = plt.subplots(figsize=(8, 8))

patches, texts, pcts = ax.pie(
    latest_covid19['Percentage'], labels=latest_covid19['Country/Region'], autopct='%.2f%%',
    wedgeprops={'linewidth': 2.0, 'edgecolor': 'white'},
    textprops={'size': '12'},
    startangle=90, pctdistance=0.75)

for i, patch in enumerate(patches):
    texts[i].set_color(patch.get_facecolor())

plt.setp(pcts, color='white')
ax.set_title('Top COVID-19 Affected Countries', fontsize=18)
plt.tight_layout()

plt.show()

### 8.2 พิจารณาเฉพาะประเทศแคนาดา (รวมกัน 9 วันของเดือนล่าสุด Mar 2023) ให้สร้างกราฟแสดงจำนวน ผู้เสียชีวิตสะสมเรียงลำดับจากมากไปน้อย โดยให้เลือกแสดงเฉพาะรัฐที่มีผู้เสียชีวิตหลักหมืนขึ้นไปเท่านั้น (4 คะแนน)

In [ ]:
day = 9
date = '3/9/2023'

canada_covid = df_covid19[df_covid19['Country/Region'] == 'Canada']

end_date = canada_covid.columns.get_loc(date)
start_date = end_date - day

date_columns = canada_covid.columns[start_date:end_date + 1]

canada_covid = canada_covid[['Province/State', 'Country/Region'] + list(date_columns)]
canada_covid.drop(canada_covid[canada_covid[date] < 10000].index, inplace=True)
canada_covid['Total'] = canada_covid[date] - canada_covid[date_columns[0]]
canada_covid.sort_values(by='Total', ascending=False, inplace=True)

กราฟ

In [ ]:
plt.figure(figsize=(12, 6))

bars = plt.bar(canada_covid['Province/State'], canada_covid['Total'], color='skyblue')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:,.0f}', ha='center', va='bottom', fontsize=9, color='black')

plt.grid(axis='y', alpha=0.5)
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: '{:,.0f}'.format(x)))
plt.xticks(rotation=45)
plt.title('Total Deaths in Canada by Province/State')
plt.xlabel('Province/State in Canada', labelpad=30)
plt.ylabel('Total Deaths', labelpad=40)
plt.tight_layout()

plt.show()

### 8.3 หาจำนวนรวมผู้เสียชีวิตที่เกิดขึ้นในแต่ละเดือน ของแต่ละปี และสร้าง Bubble Chart โดยกำหนดให้แนวนอนเป็นปี แนวตั้ง เป็นเดือน ค่าในแต่ละจุดเป็นจำนวนผู้เสียชีวิตในเดือนกับปีนั้นๆ (3 คะแนน)

In [ ]:
covid = df_covid19.iloc[:, 4:]
covid = covid.sum().reset_index(name='Total cases')
covid.rename(columns={'index': 'Date'}, inplace=True)
covid['Date'] = pd.to_datetime(covid['Date'], format='%m/%d/%y', errors='coerce')

covid['Year-Month'] = covid['Date'].dt.to_period('M')
covid = covid.groupby('Year-Month')['Total cases'].agg([('Max', 'max')]).reset_index()
covid['Max_Diff'] = covid['Max'].diff()
covid.loc[0, 'Max_Diff'] = covid.loc[0, 'Max']

covid['Month'] = covid['Year-Month'].dt.month 
covid['Year'] = covid['Year-Month'].dt.year

กราฟ

In [ ]:
plt.figure(figsize=(8,6))

sns.scatterplot(data=covid, x='Year', y='Month', size='Max_Diff', sizes=(20,500), alpha=0.7, edgecolor='black', linewidths=0, hue='Max_Diff', palette='RdYlGn_r')

x_ticks = np.arange(min(covid['Year']), max(covid['Year']) + 1)
y_ticks = np.arange(1, 13)
plt.legend(title='Total Cases', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(x_ticks)
plt.yticks(y_ticks)
plt.title('COVID-19 Cases by Month and Year')
plt.xlabel('Year', labelpad=30)
plt.ylabel('Month', labelpad=40)
plt.tight_layout()

plt.show()

## 9. พิจารณาข้อมูลยอดเหตุการณ์อุบัติเหตุยานพาหนะ accidents.xlsx ที่ให้มาและเข้าถึงเว็บ https://en.wikipedia.org/wiki/Provinces_of_Thailand เพื่อใช้สำหรับแปลงชื่อจังหวัดภาษาไทยให้เป็นอังกฤษ และนำพื้นที่ (Area) และจำนวน ประชากร (ปี 2022) มาใช้งานร่วมกับข้อมูลอุบัติเหตุ

In [ ]:
province_url = 'https://en.wikipedia.org/wiki/Provinces_of_Thailand'
province_table = pd.read_html(province_url)
province_table = province_table[2]

accidents = df_accidents.copy()
accidents['Province'] = accidents['Province'].map(province_table.set_index('Name (in Thai)')['Namesake town/city'])

### 9.1 จำนวนครั้งการเกิดอุบัตัเหตุในเขตกรุงเทพมหานคร โดยเรียงลำดับตามจำนวนครั้งการเกิดสำหรับประเภทยานพาหนะ (Vehicle) จากมากไปน้อย โดยนำเสนอเป็นทั้งตัวเลข และ กราฟที่เหมาะสม (3 + 2 = 5 คะแนน)

In [ ]:
accidents_in_bkk = accidents[accidents['Province'] == 'Bangkok']

accidents_in_bkk = accidents_in_bkk.groupby('Vehicle').size().reset_index(name='Accident Count')
accidents_in_bkk.sort_values(by='Accident Count', ascending=False, inplace=True)
accidents_in_bkk

กราฟ

In [ ]:
plt.rcParams['font.family'] = 'Angsana New'
plt.rcParams['font.size'] = 20
plt.figure(figsize=(20,10))
plt.barh(accidents_in_bkk['Vehicle'], accidents_in_bkk['Accident Count'], color='skyblue')

for i, v in enumerate(accidents_in_bkk['Accident Count']):
    plt.text(v, i, str(v)+ ' คัน', fontsize=16)

plt.xlabel('Vehicle', labelpad=30)
plt.ylabel('Accident Count', labelpad=40)
plt.title('Accident Count in Bangkok by Vehicle')
plt.tight_layout()

plt.show()

### 9.2 นำเสนอตารางสรุปข้อมูลจำนวนประชากร พื้นที่ ความหนาแน่น จำนวนครั้งอุบัติเหตุ จำนวนครั้งอุบัติเหตุต่อจำนวนประชากรหนึ่งแสนคน และ จำนวนครั้งอุบัติเหตุต่อพื้นที่ ของแต่ละจังหวัด โดยเรียงลำดับตามคอลัมน์จำนวนครั้งอุบัติเหตุต่อจำนวนประชากรหนึ่งแสนคน (Acct/Pop(100000 คน)) ดังตัวอย่างต่อไปนี้ (6 คะแนน)

In [ ]:
accidents_count = accidents.groupby(['Province']).size().reset_index(name='Accident Count')

accidents_detail = pd.merge(province_table, accidents_count, left_on='Namesake town/city', right_on='Province')
accidents_detail = accidents_detail[['Province', 'Population (December 2022)[1]', 'Area (km2)[2]', 'Population density', 'Accident Count']]
accidents_detail['(Acct/Pop(100000))'] = accidents_detail['Accident Count'] / accidents_detail['Population (December 2022)[1]'] * 100000
accidents_detail['Acct/Area(km2)'] = accidents_detail['Accident Count'] / accidents_detail['Area (km2)[2]']

accidents_detail.rename(columns={'Population (December 2022)[1]':'Population', 'Area (km2)[2]':'Area'}, inplace=True)
pd.options.display.float_format = '{:,.2f}'.format
accidents_detail.sort_values(by='(Acct/Pop(100000))', ascending=False)

### 9.3 จากข้อย่อยที่แล้ว (9.2) ท่านคิดว่าจำนวนคร้้งอุบัติเหตุมีความสัมพันธ์กับข้อมูล {Population, Area, Density} ใด มากน้อยกว่ากันอย่างไร (3 คะแนน)

**รูปภาพอ้างอิงแสดงความสัมพันธ์

![image.png](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*nElSDIp1jloJiqWX7hniaQ.png)

#### Accident/Population

กราฟแสดงความสัมพันธ์ระหว่างจำนวนประชากรกับจำนวนอุบัติเหตุ

In [ ]:
sns.jointplot(data=accidents_detail, x='Accident Count', y='Population', kind='reg', height=9)
r, _ = stats.pearsonr(accidents_detail['Accident Count'], accidents_detail['Population'])

print(f'ค่า r (correlation coefficient) คือ: {r}')

จากกราฟแสดงความสัมพันธ์ระหว่างจำนวนประชากรกับจำนวนอุบัติเหตุซึ่งมีค่า correlation coefficient คือ: 0.8227221197691654 แสดงให้เห็นว่า จำนวนประชากรกับจำนวนการเกิดอุบัติเหตุนั้นมีความสัมพันธ์กันเป็นอย่างมาก

#### Accident/Area

กราฟแสดงความสัมพันธ์ระหว่างพื้นที่กับจำนวนอุบัติเหตุ

In [ ]:
sns.jointplot(data=accidents_detail, x='Accident Count', y='Area', kind='reg', height=9)
r, _ = stats.pearsonr(accidents_detail['Accident Count'], accidents_detail['Area'])

print(f'ค่า r (correlation coefficient) คือ: {r}')

จากกราฟแสดงความสัมพันธ์ระหว่างพื้นที่กับจำนวนอุบัติเหตุซึ่งมีค่า correlation coefficient คือ: 0.4396194049245709 แสดงให้เห็นว่า พื้นที่กับจำนวนการเกิดอุบัติเหตุนั้นมีความสัมพันธ์กันน้อย

#### Accident/Density

กราฟแสดงความสัมพันธ์ระหว่างความหนาแน่นของประชากรกับจำนวนอุบัติเหตุ

In [ ]:
sns.jointplot(data=accidents_detail, x='Accident Count', y='Population density', kind='reg', height=9)
r, _ = stats.pearsonr(accidents_detail['Accident Count'], accidents_detail['Population density'])
print(f'ค่า r (correlation coefficient) คือ: {r}')


จากกราฟแสดงความสัมพันธ์ระหว่างความหนาแน่นของประชากรกับจำนวนอุบัติเหตุซึ่งมีค่า correlation coefficient คือ: 0.3144337685268983 แสดงให้เห็นว่า ความหนาแน่นของประชากรกับจำนวนการเกิดอุบัติเหตุนั้นมีความสัมพันธ์กันน้อยมาก